In [1]:
# importing all needed libraries and api keys
import requests
import json
import re
import pandas as pd
import itertools

from api_resources.spotify_credentials import client_id, client_secret
from api_resources.spotify_token_request import r

{'access_token': 'BQBfyB_oBm8vPrdzXgFAzYl3d-o1Ah2Ad_RZv5luhU9p212XgUE9Dz0jr6cmI3EFXYlfVvqVbj0D8qkPC49s5vPZcEsGi2r09db3wkUShseKNhIZ4H4',
 'expires_in': 3600,
 'token_type': 'Bearer'}


In [2]:
# spotify information and headers for future api use
spotify_api_key = r['access_token']
spotify_headers = {
    'Authorization': 'Bearer {}'.format(spotify_api_key)
}

In [3]:
# reading id csv and converting into a dictionary
csv_df = pd.read_csv('resources/all_top100songs_by_year_final.csv')

data_dictionary = {}

for x in range(0, 4500):
    data_dictionary[x] = {}
    data_dictionary[x]['title'] = csv_df['title'][x]
    data_dictionary[x]['artist'] = csv_df['artist'][x]
    data_dictionary[x]['id'] = csv_df['id'][x]

data_dictionary

{0: {'title': 'Lose Control',
  'artist': 'Teddy Swims',
  'id': '17phhZDn6oGtzMe56NuWvj'},
 1: {'title': 'Stick Season',
  'artist': 'Noah Kahan',
  'id': '0mflMxspEfB0VbI1kyLiAv'},
 2: {'title': 'Beautiful Things',
  'artist': 'Benson Boone',
  'id': '3xkHsmpQCBMytMJNiDf3Ii'},
 3: {'title': 'Houdini', 'artist': 'Dua Lipa', 'id': '6D8y7Bck8h11byRY88Pt2z'},
 4: {'title': 'I Remember Everything',
  'artist': 'Zach Bryan Featuring Kacey Musgraves',
  'id': '4KULAymBBJcPRpk1yO4dOG'},
 5: {'title': 'Saturn', 'artist': 'SZA', 'id': '1bjeWoagtHmUKputLVyDxQ'},
 6: {'title': 'Feather',
  'artist': 'Sabrina Carpenter',
  'id': '2Zo1PcszsT9WQ0ANntJbID'},
 7: {'title': 'Austin', 'artist': 'Dasha', 'id': '2uqYupMHANxnwgeiXTZXzd'},
 8: {'title': 'Lovin On Me',
  'artist': 'Jack Harlow',
  'id': '4xhsWYTOGcal8zt0J161CU'},
 9: {'title': 'Too Sweet', 'artist': 'Hozier', 'id': '76JKIsdKrAfWUMjaA0u7v5'},
 10: {'title': 'Like That',
  'artist': 'Future, Metro Boomin & Kendrick Lamar',
  'id': '2tudvzsrR5

In [4]:
# creating a function to call on spotify track analysis for a certain range of data; this is to prevent rate limit issues when calling on spotify api
# the relevant data collected through this function are the song's duration, loudness, tempo, time signature, and key
audio_analysis = {}
audio_analysis_df = pd.DataFrame()

def audio_analyze(start, end):

    global audio_analysis, audio_analysis_df, data_dictionary
    
    for x in itertools.islice(data_dictionary.keys(), start, end):
        song = data_dictionary[x]
        id = song['id']
        
        try:
            audio_analysis_url = f"https://api.spotify.com/v1/audio-analysis/{id}"
            
            audio_analysis_response = requests.get(audio_analysis_url, headers = spotify_headers).json()
            
            audio_analysis["index"] = x
            audio_analysis["title"] = song['title']
            
            audio_analysis["duration"] = audio_analysis_response["track"]["duration"]
            audio_analysis["loudness"] = audio_analysis_response["track"]["loudness"]
            audio_analysis["tempo"] = audio_analysis_response["track"]["tempo"]
            audio_analysis["time_signature"] = audio_analysis_response["track"]["time_signature"]
            audio_analysis["key"] = audio_analysis_response["track"]["key"]
    
            song_df = pd.DataFrame(audio_analysis, index = [int(x)])
            audio_analysis_df = pd.concat([audio_analysis_df, song_df])
    
        except Exception as e:
            print(f"{song['title']} was not analyzed")
            print(f"{e}, {type(e)}")


In [5]:
# creating a function to call on spotify track audio features for a certain range of data; this is to prevent rate limit issues when calling on spotify api
# the relevant data collected through this function are the song's danceability, energy, and valence, all calculated by spotify
audio_features_dict = {}
audio_features_df = pd.DataFrame()

def audio_features(start, end):

    global audio_features_dict, audio_features_df, data_dictionary

    for x in itertools.islice(data_dictionary.keys(), start, end):
        song = data_dictionary[x]
        id = song['id']
        
        try:
            audio_features_url = f"https://api.spotify.com/v1/audio-features/{id}"
            
            audio_features_response = requests.get(audio_features_url, headers = spotify_headers).json()

            audio_features_dict["index"] = x
            audio_features_dict["title"] = song['title']
            
            audio_features_dict["danceability"] = audio_features_response["danceability"]
            audio_features_dict["energy"] = audio_features_response["energy"]
            audio_features_dict["valence"] = audio_features_response["valence"]
    
            song_df = pd.DataFrame(audio_features_dict, index = [int(x)])
            audio_features_df = pd.concat([audio_features_df, song_df])
    
        except Exception as e:
            print(f"{song['title']} was not analyzed")
            print(f"{e}, {type(e)}")


In [ ]:
# running audio analyze function on 1000 songs at a time to avoid rate limiting

In [ ]:
audio_analyze(0, 1000)

In [ ]:
audio_analyze(0, 1000)

In [ ]:
audio_analyze(1000, 2000)

In [ ]:
audio_analyze(2000, 3000)

In [ ]:
audio_analyze(3000, 4000)

In [ ]:
audio_analyze(4000, 4501)

In [ ]:
# resetting index to merge the top 100 songs per year csv dataframe with the audio analysis dataframe on the index
csv_df.reset_index(inplace=True)

In [ ]:
# merging and exporting the new dataframe with duration, loudness, tempo, time signature, and key
master_p1_df = pd.merge(csv_df, audio_analysis_df, how = 'outer', on = 'index')
master_p1_df = master_p1_df.drop(columns = ['title_y'])
master_p1_df = master_p1_df.rename(columns = {"title_x": "title"})

master_p1_df.to_csv('master_data_part_1.csv', index = False)

In [ ]:
# running audio features function on 100 songs at a time to avoid rate limiting

In [ ]:
audio_features(0, 100)

In [ ]:
audio_features(100, 200)

In [ ]:
audio_features(200, 300)

In [ ]:
audio_features(300, 400)

In [ ]:
audio_features(400, 500)

In [ ]:
audio_features(500, 600)

In [ ]:
audio_features(600, 700)

In [ ]:
audio_features(700, 800)

In [ ]:
audio_features(800, 900)

In [ ]:
audio_features(900, 1000)

In [ ]:
audio_features(1000, 1100)

In [ ]:
audio_features(1100, 1200)

In [ ]:
audio_features(1200, 1300)

In [ ]:
audio_features(1300, 1400)

In [ ]:
audio_features(1400, 1500)

In [ ]:
audio_features(1500, 1600)

In [ ]:
audio_features(1600, 1700)

In [ ]:
audio_features(1700, 1800)

In [ ]:
audio_features(1800, 1900)

In [ ]:
audio_features(1900, 2000)

In [ ]:
audio_features(2000, 2100)

In [ ]:
audio_features(2100, 2200)

In [ ]:
audio_features(2200, 2300) 

In [ ]:
audio_features(2300, 2400)

In [ ]:
audio_features(2400, 2500)

In [ ]:
audio_features(2500, 2600)

In [ ]:
audio_features(2600, 2700)

In [ ]:
audio_features(2700, 2800)

In [ ]:
audio_features(2800, 2900)

In [ ]:
audio_features(2900, 3000)

In [ ]:
audio_features(3000, 3100)

In [ ]:
audio_features(3100, 3200)

In [ ]:
audio_features(3200, 3300)

In [ ]:
audio_features(3300, 3400)

In [ ]:
audio_features(3400, 3500)

In [ ]:
audio_features(3500, 3600)

In [ ]:
audio_features(3600, 3700)

In [ ]:
audio_features(3700, 3800)

In [ ]:
audio_features(3800, 3900)

In [ ]:
audio_features(3900, 4000)

In [ ]:
audio_features(4000, 4100)

In [ ]:
audio_features(4100, 4200)

In [ ]:
audio_features(4200, 4300)

In [ ]:
audio_features(4300, 4400)

In [ ]:
audio_features(4400, 4500)

In [ ]:
# combining audio analysis and audio features dataframe into a complete final dataframe and exporting to csv
part_1_df = pd.read_csv('master_data_part_1.csv')
master_df = pd.merge(part_1_df, audio_features_df, how = 'outer', on = 'index')

master_df = master_df.drop(columns = ['title_y'])
master_df = master_df.rename(columns = {"title_x": "title"})

master_df.to_csv('master_data_final.csv', index = False)